In [1]:
import numpy as np
import pandas as pd
import sympy as sp
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima import auto_arima

In [12]:
S=pd.read_excel(io=r'../data/402-dataset.xlsx',sheet_name='Supply',usecols="C:IH").values
F=pd.read_excel(io=r'../data/8-dataset.xlsx',sheet_name='Loss',usecols="B:IG").values
MT=np.diag(1./(pd.read_excel(io=r'../data/402-dataset.xlsx',sheet_name='Supply',usecols="B").values.flatten()))
S=np.dot(MT,S)
supply=np.zeros((402,24,10))
forwa=np.zeros((8,24,10))
for i in range(402):
    supply[i,:,:]=S[i,:].reshape(24,10)
for j in range(8):
    forwa[j,:,:]=F[j,:].reshape(24,10)

In [16]:
def GM_11(x):
    n=len(x)
    lamda=x[:-1]/x[1:]
    rang=[lamda.min(), lamda.max()]
    theta=[np.exp(-2/(n+1)),np.exp(2/(n+1))]
    x0=x
    if theta[0]<=rang[0] and rang[1]<=theta[1]:
        c=0
    else:
        while True:
            x0=x0+1
            ratio=x0[:-1]/x0[1:]
            if theta[0]<=ratio.min() and ratio.max()<=theta[1]:
                break
        c=x0[0]-x[0]
    x=x+c
    x1=np.cumsum(x)
    z=0.5*(x1[1:]+x1[:-1])
    B=np.vstack([-x[1:],-z,np.ones(n-1)]).T
    u=np.linalg.pinv(B)@np.diff(x)
    sp.var('t'); sp.var('xx',cls=sp.Function)  
    eq=xx(t).diff(t,2)+u[0]*xx(t).diff(t)+u[1]*xx(t)-u[2]
    s=sp.dsolve(eq,ics={x(0):x[0],xx(5):x1[-1]})
    xt=s.args[1]
    fxt=sp.lambdify(t,xt,'numpy') 
    fo1=fxt(np.arange(n))
    fo=np.hstack([x[0],np.diff(fo1)]) 
    fo=fo-c
    x=x-c
    epsilon=x.T-fo
    fo=fo[-1]
    return fo,epsilon

In [14]:
def ARIMA(x):
    x0=x
    di=0
    while True:
        if kpss(x0,regression='ct')[0]>kpss(x0,regression='ct')[3]['5%']:
            x0=np.diff(x0)
            di=di+1
        else:
            break
    model = auto_arima(x0, start_p=1, d=0, start_q=1,max_d=0,
                       seasonal=False, test='adf',stepwise=False,stationary=True,
                       error_action='ignore',transparams=False,
                       information_criterion='aic',
                       njob=-1, trace=True, suppress_warnings=True)
    forecast = model.predict(n_periods=len(x)+1-len(x0))
    fo=np.cumsum(forecast)
    x_fo=x[-1]+fo[-1]
    return x_fo
#y1=np.array([1.5539, 1.4642, 2.1018, 1.9554, 2.1018, 2.2294,1.9507, 1.6248, 1.8232, 1.7759])
#y2=np.array([1.639,2.0404,2.645,2.5033,2.3333,2.4372,2.2577,2.201,1.9649,1.6295])

In [ ]:
S_fo=np.zeros((24,402))
F_fo=np.zeros((8,402))
for k in range(402):
    fc=np.zeros(24)
    for l in range(24):
        fo,epsilon=GM_11(supply[k,l,:])
        epsilon=ARIMA(epsilon)
        fc=fo+epsilon[-1]
    S_fo[:,i]=fc
for m in range(8):
    fc=np.zeros(24)
    for n in range(24):
        fo,epsilon=GM_11(supply[m,n,:])
        epsilon=ARIMA(epsilon)
        fc=fo+epsilon[-1]
    F_fo[:,i]=fc